<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Variational_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import(
    layers,
    models,
    datasets,
    callbacks,
    losses,
    optimizers,
    metrics,
)

from scipy.stats import norm



#0. Parameters

In [2]:
IMAGE_SIZE = 32
BATCH_SIZE = 100
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM  = 2
EPOCHS = 5
BETA = 500

#1. Prepare the data

In [3]:
#Load the data
(x_train, y_train) , (x_test, y_test) = datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
#preprocess the data

def preprocess(imgs):
  """
  Normalize and reshape the images
  """
  imgs = imgs.astype("float32") / 255.0
  imgs = np.pad(imgs , ((0,0) , (2,2) , (2,2)), constant_values = 0.0)
  imgs = np.expand_dims(imgs, -1)
  return imgs

x_train = preprocess(x_train)
x_test = preprocess(x_test)

In [5]:
#Show some items of clothing from the training set
display(x_train)


array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

#2. Build the variational autoencoder

In [6]:
class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean , z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape = (batch, dim))
    return z_mean +tf.exp(0.5 * z_log_var) * epsilon

In [7]:
#Encoder
encoder_input = layers.input(
    shape = (IMAGE_SIZE , IMAGE_SIZE, 1), name = "encoder_input"
)
x = layers.Conv2D(32, (3,3) , strides = 2, activation = "relu" , padding = "same")(
    encoder_input
)
x = layers.Conv2D(64, (3,3) , strides = 2, activation =  "relu" , padding = "same")(x)
x = layers.Conv2D(128, (3,3), strides = 2, activation = "relu" , padding = "same")(x)
shape_before_flattening = K.int_shape(x)[1:] # the decoder will need this!

x = layers.Flatten()(x)
z_mean = layers.Dense(EMBEDDING_DIM, name = "z_mean")(x)
z_log_var = layers.Dense(EMBEDDING_DIM , name = "z_log_ver")(x)
z = Sampling()([z_mean , z_log_var])

encoder = models.Model(encoder_input , [z_mean, z_log_var, z] , name=  "encoder")
encoder.summary()


AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'input'

In [8]:
#Decoder
decoder_input = layers.Input(shape = (EMBEDDING_DIM ,), name = "decoder_input")
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(
    128, (3,3) , strides = 2, activation = "relu", padding = "same"
)(x)
x = layers.Conv2DTranspose(
    32, (3,3) , strides = 2, activation = "relu", padding = "same"
)(x)
decoder_output = layers.Conv2D(
    1,
    (3,3),
    strides = 1,
    activation = "sigmoid",
    padding = "same",
    name = "decoder_output",
)(x)

decoder = models.Model(decoder_input , decoder_output)
decoder_summary()


NameError: name 'shape_before_flattening' is not defined

In [9]:
class VAE(models.Model):
  def __init__(self, encoder, decoder, **kwargs):
    super(VAE, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder
    self.total_loss_tracker = metrics.Mean(name = "total_loss")
    self.reconstruction_loss_tracker = metrics.Mean(
        name = "reconstruction_loss"
    )
    self.kl_loss_tracker = metrics.Mean(name = "kl_loss")

  @property
  def metrics(self):
    return[
        self.total_loss_tracker,
        self.reconstruction_loss_tracker,
        self.kl_loss_tracker,
    ]

  def call(self, inputs):
    """Call the model on a particular input"""
    z_mean , z_log_var , z = encoder(inputs)
    reconstruction = decoder(z)
    return z_mean , z_log_var, reconstruction


  def train_step(self, data):
    """Step run during training."""
    with tf.GradientTape() as tape:
      z_mean, z_log_var , reconstruction = self(data)
      reconstruction_loss = tf.reduce_mean(
          BETA
          * losses.binary_crossentropy(
              data, reconstruction, axis = (1,2,3)
          )
      )
      kl_loss = tf.reduce_mean(
          tf.reduce_sum(
              -0.5
              * (1 + z_log_var - tf.square(z_mean)  - tf.exp(z_log_var))
              axis = 1,
          )
      )
      total_loss = reconstruction_loss  + kl_loss

    grads = tape.gradient(total_loss , self.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

    self.total_loss_tracker.update_state(total_loss)
    self.reconstruction_loss_tracker.update_state(reconstruction_loss)
    self.kl_loss_tracker.update_state(kl_loss)

    return {m.name : m.result() for m in self.metrics}

  def test_step(self, data):
    """Step run during validation."""
    if isinstance(data, tuple):
      data = data[0]

    z_mean , z_log_var , reconstruction = self(data)
    reconstruction_loss = tf.reduce_mean(
        BETA
        * losses.binary_crossentropy(data, reconstruction, axis = (1, 2, 3))
    )
    kl_loss = tf.reduce_mean(
        tf.reduce_sum(
            -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)),
            axis = 1,
        )
    )
    total_loss = reconstruction_loss + kl_loss

    return {
        "loss":total_loss,
        "reconstruction_loss":reconstruction_loss,
        "kl_loss":kl_loss,
    }


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-9-f5721a3c254d>, line 39)

In [10]:
#Create a variational autoencoder
vae = VAE(encoder, decoeder)

NameError: name 'VAE' is not defined

#3. Train the variational autoencoder

In [11]:
#compile the variational autoencoder
optimizer = optimizers.Adam(leanring_rate = 0.0005)
vae.compile(optimizer = optimizer)

TypeError: leanring_rate is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [12]:
#Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint",
    save_weights_only = False,
    save_freq = "epoch",
    monitor = "loss",
    mode = "min",
    save_best_only = True,
    versbose = 0,
)
tensorboard_callback = callbacks.TensorBoard(lod_dir = "./logs")


ValueError: Unrecognized arguments in `TensorBoard` Callback: {'lod_dir'}. Supported kwargs are: {'embeddings_data', 'batch_size', 'write_grads', 'embeddings_layer_names'}

In [13]:
vae.fit(
    x_train,
    epochs  = EPOCHS,
    batch_size = BATCH_SIZE ,
    shuffle = True,
    validation_data = (x_test , x_test),
    callbacks = [model_checkpoint_callback, tensorboard_callback],
)


NameError: name 'vae' is not defined

In [ ]:
#save the final models
vae.save("./models/vae")
encoder.save("./models/encoer")
decoder.save("./models/decoder")

#3. Reconstruct using the variational autoencoder

In [ ]:
#Select a subset of the test set
n_to_predict = 5000
example_images = x_test[:n_to_predict]
example_labels = y_test[:n_to_predict]


In [ ]:
#Create autoencoder predictions and display
z_mean, z_log_var , reconstruction = vae.predict(example_images)
print("Example real clothing items")
display(example_images)
print("reconstructions")
display(reconstructions)


#4. Embed using the encoder

In [ ]:
#Encode the exapmle images
z_mean, z_var , z = encoder(example_images)

In [ ]:
#some exapmle of the embeddings
print(z[:10])

In [ ]:
#Show the encoded points in 2D space
figsize = 8

plt.figure(figsize = (figsize, figsize))
plt.scatter(z[:, 0]  z[:, 1] , c = "black" , alpha = 0.5 , s= 3)
plt.show()


#5. Generate using the decoder

In [ ]:
#sample some points in the latent space , from the standard normal distribution
grid_with , grid_height = (6, 3)
z_sample = np.random.normal(size = (grid_width * grid_height, 2))

In [ ]:
#Decode the smapled points
reconstructions = decoder.predict(z_sample)


In [ ]:
#Convert orignal embeddings and sampled embeddings to p-value
p = norm.cdf(z)
p_sample = norm.cdf(z_sample)

In [ ]:
#Draw a plot of
figsize = 8
plt.figure(figsize = (figsize , figsize))

# ... the orignal embeddings ...
plt.scatter(z[:, 0] , z[: ,1 ], c = "black" , alpha = 0.5, s = 2)

# ... and the newly generated points in the latent space
plt.scatter(z_sample[:, 0] , z_sample[:,1] , c = "#00B0F0" , alpha = 1, s = 40)
plt.show()

#Add underneath a grid of the decoded images
fig = plt.figure(figsize = (figsize, grid_height * 2))
fig.subplots_adjust(hspace = 0.4 , wspace = 0.4)

for i in range(grid_width * grid_height):
  ax = fig.add_subplot(grid_height , grid_width , 1 + i)
  ax.axis("off")
  ax.text(
      0.5,
      -0.35,
      str(np.round(z_sample[i, :] , 1)),
      fontsize = 10,
      ha = "center",
      transform = ax.transAxes,
  )
  ax.imshow(reconstructions[i, :, :] , cmap = "Greys")

#6. Explore the latent space

In [ ]:
#Color the embeddings by their label (clothing type - see table)
figsize = 8
fig = plt.figure(figsize = (figsize * 2, figsize))
ax = fig.add_subplot(1, 2,1)
plot_1 = ax.scatter(
    z[:, 0] , z[:, 1] , cmap = "rainbow", c = example_labels , alpha = 0.8, s = 3
)
plt.colorbar(plot_1)
ax = fig.add_subplot(1,2,2)
plot_2 = ax.scatter(
    p[:, 0] , p[: ,1 ] , cmap = "rainbow" , c = example_labels , alpha = 0.8 , s = 3
)
plt.show()


In [ ]:
#color the embeddings by their label (clothing type - )
figsize = 12
grid_size = 15
plt.figure(figsize = (figsize , figsize))
plt.scatter(
    p[:, 0],p[:,1], cmap = "rainbow" , c = example_labels, alpha = 0.8, s = 300
)
plt.colorbar()

x = norm.ppf(np.linspace(0, 1 , grid_size))
y = norm.ppf(np.linspace(1,0 , grid_size))
xv, yv = np.meshgrid(x, y)
xv = xv.flatten()
yv = yv.flatten()
grid = np.arry(list(zip(xv, yv)))

reconstructions = decoder.predict(grid)

plt.show()

fig = plt.figure(figsize= (figsize, figsize))
fig.subplots_adjust(hspace = 0.4, wspace = 0.4)
for i in range(grid_size ** 2):
  ax = fig.add_subplot(grid_size , grid_size, i + 1)
  ax.axis("off")
  ax.imshow(reconstructions[i, :, :], cmap = "Greys")